This notebook compares the gnubg bearoff database to our calculations. It assumes that you have generated the data files with these commands

`./compute.py 15 6`

`./gnubg_to_store.py ~/gnubg/gnubg-1.06.002/`

For the latter, it should be a directory where you have the source of gnubg downloaded and made


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import board
import gnubg_interface
import strategy

In [3]:
!ls -l data/

total 50260
-rw-rw-r-- 1 pfr pfr 27284736 Mar 23 22:29 bgend_store_15_6.hdf5
-rw-rw-r-- 1 pfr pfr    37712 Mar 24 19:17 bgend_store_6_3.hdf5
-rw-rw-r-- 1 pfr pfr 24134504 Mar 22 21:47 gnubg_store_15_6.hdf5


In [4]:
%%time
our_store = strategy.DistributionStore.load_hdf5('data/bgend_store_15_6.hdf5')
gnubg_store = strategy.DistributionStore.load_hdf5('data/gnubg_store_15_6.hdf5')

CPU times: user 22.6 s, sys: 489 ms, total: 23.1 s
Wall time: 31.6 s


In [5]:
print(len(our_store.distribution_map))
print(len(gnubg_store.distribution_map))
assert(len(our_store.distribution_map) == len(gnubg_store.distribution_map))

54264
54264


In [ ]:
# The 1e-4 threshold is because of the limited precision from going through the text parsing from gnubg
total_boards = 0
total_correct = 0
for board_id in our_store.distribution_map:
    total_boards += 1
    ours_orig = our_store.distribution_map[board_id]
    ours = ours_orig.trim_low_prob(1e-5)
    theirs = gnubg_store.distribution_map[board_id]
    if len(ours) != len(theirs):
        print('Board ID {}, Position ID {}, different lengths\nours(orig): {}\nours(trim): {}\ntheirs  : {}'
              .format(board_id, gnubg_interface.board_id_to_gnubg_id_str(our_store.config, board_id), ours_orig, ours, theirs))
        continue

    if not np.allclose(ours.dist, theirs.dist, atol=1e-4):
        print('Board ID {}, Position ID {}, mismatch\nours(orig): {}\nours(trim): {}\ntheirs  : {}'
              .format(board_id, gnubg_interface.board_id_to_gnubg_id_str(our_store.config, board_id), ours_orig, ours, theirs))
        continue
        
    total_correct += 1
    
print('{}/{} correct ({:.2f}%)'.format(total_correct, total_boards, total_correct / total_boards))